# PySpark 종합 실습 Notebook

**환경**: Python 3.9~3.10, OpenJDK 11, PySpark 3.x, MongoDB
**목적**: 데이터 처리, 머신러닝, MongoDB 연동 실습

In [3]:
! pip install --upgrade pyspark==3.5

  Using cached pyspark-3.5.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.1
    Uninstalling pyspark-4.0.1:
      Successfully uninstalled pyspark-4.0.1



In [4]:
! pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: c:\users\user\anaconda3\lib\site-packages
Requires: py4j
Required-by: 


In [15]:
# --------------------------------
# 1. SparkSession 생성
# --------------------------------
from pyspark.sql import SparkSession

# SparkSession 생성 및 MongoDB 연결 설정
spark = SparkSession.builder \
    .appName("PySparkFullExample") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/test_db.output") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/test_db.input") \
    .getOrCreate()

print("SparkSession created successfully")

SparkSession created successfully


실행이 느린 이유 상세

JVM 시작 시간

- PySpark는 내부적으로 Java를 사용

- JVM 실행 + SparkContext 생성 → 초기화 시간이 발생

리소스 초기화

- 로컬 모드라도 Spark는 Executor, Driver, Shuffle, Logging 등 내부 구성요소를 초기화

- 특히 메모리 부족, CPU 코어 부족 시 더 느림

환경 설정

- Windows 환경에서 JVM PATH, Hadoop 버전 확인, 방화벽 등 환경 설정 문제

- SparkSession builder 옵션이 많거나 MongoDB 등 외부 커넥터 연동 시 초기화 지연

In [19]:
# --------------------------------
# 2. DataFrame 생성 및 데이터 처리
# --------------------------------
# from pyspark.sql.functions import col, when

# # 샘플 데이터 생성
# data = [("Alice", 34, 5000), ("Bob", 45, 4000), ("Cathy", 29, 3000), ("David", 30, 2000)]
# columns = ["name", "age", "salary"]

# df = spark.createDataFrame(data, columns)
# df.show(5)

# # 결측치 처리 (age 컬럼 결측치는 30으로 대체)
# # df = df.fillna({"age": 30})


# 2. 샘플 데이터 생성 (작은 데이터로 테스트)
data = [("Alice", 34, 5000),
        ("Bob", None, 4000),
        ("Cathy", 29, 3000),
        ("David", None, None)]
columns = ["name", "age", "salary"]

df = spark.createDataFrame(data, columns)

# 3. 결측치 처리
# age 결측치는 30으로, salary 결측치는 0으로 채움
df_clean = df.fillna({"age": 30, "salary": 1000})

# 4. 컬럼 타입 확인
df_clean.printSchema()

# 5. 안전하게 일부 데이터만 출력
print("Showing first 10 rows:")
df_clean.show(1)

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = false)
 |-- salary: long (nullable = false)

Showing first 10 rows:


Py4JJavaError: An error occurred while calling o371.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10) (C-Dragon executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(R

In [ ]:
# --------------------------------
# 3. CSV 파일 읽기 및 기본 통계
# --------------------------------
# CSV 파일 예: loan.csv
df_csv = spark.read.csv("loan.csv", header=True, inferSchema=True)
df_csv.show(5)

# 기본 통계 확인
df_csv.describe().show()

In [ ]:
# --------------------------------
# 4. 머신러닝 적용 (Linear Regression)
# --------------------------------
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# feature 벡터 생성 (age, salary 사용)
assembler = VectorAssembler(inputCols=["age", "salary"], outputCol="features")
df_ml = assembler.transform(df_filtered)

# label은 salary 컬럼 사용
df_ml = df_ml.withColumnRenamed("salary", "label")

# Linear Regression 모델 학습
lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(df_ml)

# 모델 평가
print("R2 score:", lr_model.summary.r2)
print("Coefficients:", lr_model.coefficients)
print("Intercept:", lr_model.intercept)

In [ ]:
# --------------------------------
# 5. MongoDB 연동
# --------------------------------
# MongoDB에 DataFrame 저장
df_filtered.write.format("mongo") \
    .mode("overwrite") \
    .option("uri", "mongodb://127.0.0.1:27017/test_db.output") \
    .save()

# MongoDB에서 읽기
df_mongo = spark.read.format("mongo") \
    .option("uri", "mongodb://127.0.0.1:27017/test_db.output") \
    .load()

df_mongo.show()

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# SparkSession 생성
spark = SparkSession.builder \
    .appName("PySpark CSV Save Example") \
    .master("local[*]") \
    .getOrCreate()

# CSV 데이터 로드
df = spark.read.csv("./data/Subjects.csv", header=True, inferSchema=True)
df.show()

+-----+----+---+---+----+-------+
|class|name|kor|eng|math|science|
+-----+----+---+---+----+-------+
|    1| aaa| 67| 87|  90|     98|
|    1| bbb| 45| 45|  56|     98|
|    1| ccc| 95| 59|  96|     88|
|    1| ddd| 65| 94|  89|     98|
|    1| eee| 45| 65|  78|     98|
|    1| fff| 78| 76|  98|     89|
|    2| ggg| 87| 67|  65|     56|
|    2| hhh| 89| 98|  78|     78|
|    2| iii|100| 78|  56|     65|
|    2| jjj| 99| 89|  87|     87|
|    2| kkk| 98| 45|  56|     54|
|    2| lll| 65| 89|  87|     78|
+-----+----+---+---+----+-------+



In [19]:
# 총점 컬럼 추가
df = df.withColumn("total", col("kor") + col("eng") + col("math") + col("science"))

# 총점 확인
df.show()

+-----+----+---+---+----+-------+-----+
|class|name|kor|eng|math|science|total|
+-----+----+---+---+----+-------+-----+
|    1| aaa| 67| 87|  90|     98|  342|
|    1| bbb| 45| 45|  56|     98|  244|
|    1| ccc| 95| 59|  96|     88|  338|
|    1| ddd| 65| 94|  89|     98|  346|
|    1| eee| 45| 65|  78|     98|  286|
|    1| fff| 78| 76|  98|     89|  341|
|    2| ggg| 87| 67|  65|     56|  275|
|    2| hhh| 89| 98|  78|     78|  343|
|    2| iii|100| 78|  56|     65|  299|
|    2| jjj| 99| 89|  87|     87|  362|
|    2| kkk| 98| 45|  56|     54|  253|
|    2| lll| 65| 89|  87|     78|  319|
+-----+----+---+---+----+-------+-----+



In [20]:
# 기존 폴더가 존재할 경우 덮어쓰기
output_path = "output/student_total.csv"

df.write.csv(
    output_path,   # 저장 경로
    header=True,   # 헤더 포함
    mode="overwrite"  # 기존 파일 덮어쓰기
)

print(f"CSV 파일이 저장되었습니다: {output_path}")

Py4JJavaError: An error occurred while calling o183.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# SparkSession 생성
spark = SparkSession.builder \
    .appName("PySpark CSV Save Example") \
    .master("local[*]") \
    .getOrCreate()

# 데이터 로드
df = spark.read.csv("data/Subjects.csv", header=True, inferSchema=True)

# 총점 계산
df = df.withColumn("total", col("kor") + col("eng") + col("math") + col("science"))

# CSV 저장 (기존 파일 덮어쓰기)
df.write.csv("output/student_total.csv", header=True, mode="overwrite")

print("CSV 저장 완료!")


Py4JJavaError: An error occurred while calling o216.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: